In [146]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

### Spam detection

In [70]:
#load dataset
df = pd.read_csv('spam.csv', encoding='latin-1')
df = df[['v1', 'v2']]
df = df.rename(columns={'v1': 'target', 'v2': 'text'})
df.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
# dataset size
df.shape

In [ ]:
# class proportions
df.target.value_counts(normalize=True)

In [ ]:
# Use  df_train for model training
# Use df_test as  hold-out dataset for your final model perfomance estimation.
SEED = 1337
df_train, df_test = model_selection.train_test_split(df, test_size=0.4, random_state=SEED, 
                                                     shuffle=True, stratify=df.target)
print('train size %d, test size %d' % (df_train.shape[0], df_test.shape[0]))

Stratify parameter makes a split so that the proportion of values in the sample produced will be the same as the proportion of values provided to parameter stratify.

For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% of ones, stratify=y will make sure that your random split has 25% of 0's and 75% of 1's.

In [319]:
def add_metrics(metrics, value, name):
    metrics.loc[name, 'f1'] = value

In [320]:
best_metrics = pd.DataFrame()

##### a) bag of words + Decision Tree classifier

In [321]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [322]:
# build binary feature matrix from BoW model
vec = TfidfVectorizer(lowercase=True, analyzer='word', ngram_range=(1,1), norm=None, use_idf=False, binary=True)
bow_train = vec.fit_transform(df_train.text)
bow_test = vec.transform(df_test.text)

In [323]:
# encode class labels
label_enc = LabelEncoder().fit(df_train.target)
y_train = label_enc.transform(df_train.target)

In [324]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree

In [325]:
model = tree.DecisionTreeClassifier()
model.fit(bow_train, y_train)
y_pred = model.predict(bow_test)
y_test = label_enc.transform(df_test.target)
print('test', metrics.f1_score(y_test, y_pred))

test 0.8943661971830986


In [84]:
add_metrics(best_metrics, metrics.f1_score(y_test, y_pred), 'baseline_1')

In [85]:
best_metrics

,f1
baseline_1,0.884211


***b) text stemming + bag of words + MultinomialNB***

In [326]:
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import string
import re
stemmer = SnowballStemmer("english")
regex = re.compile('[%s]' % re.escape(string.punctuation))

# basic preprocessing
# make lowercase, remove punctuation and make stemming
def text_process(text):
    text = text.lower()
    text = regex.sub(' ', text)
    text = [stemmer.stem(word) for word in text.split() if word not in stopwords.words('english')]
    return " ".join(text)


df_train['text2'] = df_train.text.apply(text_process)
df_test['text2'] = df_test.text.apply(text_process)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# build binary feature matrix from BoW model
vec = TfidfVectorizer(lowercase=True, analyzer='word', ngram_range=(1,1), norm=None, use_idf=False, binary=True)
X = vec.fit_transform(df_train.text2)
print('feature matrix shape', X.shape)

# encode class labels
label_enc = LabelEncoder().fit(df_train.target)
y_train = label_enc.transform(df_train.target)

# fit our prediction model
model = MultinomialNB(alpha=1.0)
model.fit(X, y_train)
print('train', metrics.f1_score(y_train, model.predict(X)))

# perfomance on test dataset
X_test = vec.transform(df_test.text)
y_pred = model.predict(X_test)
y_test = label_enc.transform(df_test.target)
print('test', metrics.f1_score(y_test, model.predict(X_test)))

feature matrix shape (3343, 5557)
train 0.9798206278026906
test 0.9246935201401051


In [327]:
add_metrics(best_metrics, metrics.f1_score(y_test, model.predict(X_test)), 'baseline_2')

In [328]:
best_metrics

,f1
baseline_2,0.924694


***c) text stemming + bow + Logistic Regression + grid***

In [338]:
# baseline 3
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import string
import re


stemmer = SnowballStemmer("english")
regex = re.compile('[%s]' % re.escape(string.punctuation))

# basic preprocessing
# make lowercase, remove punctuation and make stemming
def text_process(text):
    text = text.lower()
    text = regex.sub(' ', text)
    text = [stemmer.stem(word) for word in text.split() if word not in stopwords.words('english')]
    return " ".join(text)


df_train['text2'] = df_train.text.apply(text_process)
df_test['text2'] = df_test.text.apply(text_process)
df_train.head()

# build feature matrix from 3-grams
vec = TfidfVectorizer(lowercase=True, analyzer='char', ngram_range=(3,3), 
                      min_df=10, # exclude 3-grams appeared in less then 10 messages. 
                      use_idf=True)
X = vec.fit_transform(df_train.text2)
print('feature matrix shape', X.shape)

#encode labels
label_enc = LabelEncoder().fit(df_train.target)
y_train = label_enc.transform(df_train.target)

# optimize  hyperparams
grid = GridSearchCV(LogisticRegression(random_state=SEED), 
                   param_grid={'C': np.logspace(0,5,10), 
                               'penalty': ['l1', 'l2']}, 
                    scoring='f1', n_jobs=-1, cv=5, verbose=True, return_train_score=True)

grid.fit(X, y_train)
print(grid.best_params_)
print(grid.best_score_)
model = grid.best_estimator_
print('train', metrics.f1_score(y_train, model.predict(X)))

X_test = vec.transform(df_test.text2)
y_pred = model.predict(X_test)
y_test = label_enc.transform(df_test.target)
print('test', metrics.f1_score(y_test, model.predict(X_test)))

feature matrix shape (3343, 2393)
Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.7s finished


{'C': 100000.0, 'penalty': 'l2'}
0.9440632976282287
train 1.0
test 0.9636048526863085


In [334]:
add_metrics(best_metrics, metrics.f1_score(y_test, model.predict(X_test)), 'baseline_3')

In [335]:
best_metrics

,f1
baseline_2,0.924694
baseline_3,0.963605


***my) text stemming + bow + Logistic Regression + grid + composition***

In [341]:
# baseline 4
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import string
import re


stemmer = SnowballStemmer("english")
regex = re.compile('[%s]' % re.escape(string.punctuation))

# basic preprocessing
# make lowercase, remove punctuation and make stemming
def text_process(text):
    text = text.lower()
    text = regex.sub(' ', text)
    text = [stemmer.stem(word) for word in text.split() if word not in stopwords.words('english')]
    return " ".join(text)


df_train['text2'] = df_train.text.apply(text_process)
df_test['text2'] = df_test.text.apply(text_process)
df_train.head()

# build feature matrix from 3-grams
vec = TfidfVectorizer(lowercase=True, analyzer='char', ngram_range=(3,3), 
                      min_df=10, # exclude 3-grams appeared in less then 10 messages. 
                      use_idf=True)
X = vec.fit_transform(df_train.text2)
print('feature matrix shape', X.shape)

#encode labels
label_enc = LabelEncoder().fit(df_train.target)
y_train = label_enc.transform(df_train.target)

# optimize  hyperparams
grid = GridSearchCV(LogisticRegression(random_state=SEED), 
                   param_grid={'C': np.logspace(0,5,num=10), 
                               'penalty': ['l1', 'l2'],
                               'tol': [1.e-4, 1.e-3, 1.e-2, 1.e-1, 1, 2, 10, 50, 100, 1000],
                               'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
                    scoring='f1', n_jobs=-1, cv=5, verbose=10, return_train_score=True, error_score=0)

grid.fit(X, y_train)
print(grid.best_params_)
print(grid.best_score_)
model2 = grid.best_estimator_
print('train', metrics.f1_score(y_train, model2.predict(X)))

X_test = vec.transform(df_test.text2)
y_pred = model2.predict(X_test)
y_test = label_enc.transform(df_test.target)
print('test', metrics.f1_score(y_test, model2.predict(X_test)))

feature matrix shape (3343, 2393)
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits
[CV] C=1.0, penalty=l1, solver=newton-cg, tol=0.0001 .................
[CV] C=1.0, penalty=l1, solver=newton-cg, tol=0.0001 .................
[CV] C=1.0, penalty=l1, solver=newton-cg, tol=0.0001 .................


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:479: FitFailedWarning: Classifier fit failed. The score on this train-test partition for these parameters will be set to 0.000000. Details: 
ValueError('Solver newton-cg supports only l2 penalties, got l1 penalty.',)
  "Details: \n%r" % (error_score, e), FitFailedWarning)


[CV]  C=1.0, penalty=l1, solver=newton-cg, tol=0.0001, score=0, total=   0.0s


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:479: FitFailedWarning: Classifier fit failed. The score on this train-test partition for these parameters will be set to 0.000000. Details: 
ValueError('Solver newton-cg supports only l2 penalties, got l1 penalty.',)
  "Details: \n%r" % (error_score, e), FitFailedWarning)


[CV] C=1.0, penalty=l1, solver=newton-cg, tol=0.0001 .................


...

[CV]  C=100000.0, penalty=l2, solver=saga, tol=0.001, score=0.9651162790697675, total=   0.7s
[CV]  C=100000.0, penalty=l2, solver=saga, tol=1, score=0.9090909090909091, total=   0.0s
[CV]  C=100000.0, penalty=l2, solver=saga, tol=0.1, score=0.9239766081871345, total=   0.0s
[CV] C=100000.0, penalty=l2, solver=saga, tol=1 ......................
[CV] C=100000.0, penalty=l2, solver=saga, tol=1 ......................
[CV] C=100000.0, penalty=l2, solver=saga, tol=1 ......................
[CV]  C=100000.0, penalty=l2, solver=saga, tol=1, score=0.888888888888889, total=   0.0s
[CV]  C=100000.0, penalty=l2, solver=saga, tol=1, score=0.9156626506024096, total=   0.0s
[CV] C=100000.0, penalty=l2, solver=saga, tol=2 ......................
[CV]  C=100000.0, penalty=l2, solver=saga, tol=1, score=0.9411764705882353, total=   0.0s
[CV] C=100000.0, penalty=l2, solver=saga, tol=2 ......................
[CV]  C=100000.0, penalty=l2, solver=saga, tol=2, score=0.888888888888889, total=   0.0s
[CV] C=1000

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[CV]  C=27825.59402207126, penalty=l1, solver=saga, tol=0.001, score=0.9590643274853802, total=   8.8s


[Parallel(n_jobs=-1)]: Done 5000 out of 5000 | elapsed:  4.5min finished


{'C': 100000.0, 'penalty': 'l2', 'solver': 'lbfgs', 'tol': 0.001}
0.9515325177072645
train 1.0
test 0.9638554216867471


In [344]:
add_metrics(best_metrics, metrics.f1_score(y_test, model2.predict(X_test)), 'baseline_3_Logit_tuned')
best_metrics

,f1
baseline_2,0.924694
baseline_3,0.963605
baseline_3_Logit_tuned,0.963855


In [354]:
from sklearn.model_selection import RandomizedSearchCV
grid = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=SEED), 
                   param_distributions={'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
#                               'max_depth' : [int(x) for x in np.linspace(10, 110, num = 11)] + [None],
#                               'min_samples_split' : [2, 5, 10],
#                               'min_samples_leaf' : [1, 2, 4],
                               'bootstrap' : [True, False],
                               'criterion': ['entropy', 'gini'],
                               'max_features': ['sqrt','log2',None]
                               },
                    scoring='f1', n_jobs=-1, cv=5, verbose=10, return_train_score=True, error_score=0)

In [358]:
grid.fit(X, y_train)
print(grid.best_params_)
print(grid.best_score_)
model3 = grid.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=1800, max_features=sqrt, criterion=entropy, bootstrap=False 
[CV] n_estimators=1800, max_features=sqrt, criterion=entropy, bootstrap=False 
[CV] n_estimators=1800, max_features=sqrt, criterion=entropy, bootstrap=False 
[CV] n_estimators=1800, max_features=sqrt, criterion=entropy, bootstrap=False 
[CV]  n_estimators=1800, max_features=sqrt, criterion=entropy, bootstrap=False, score=0.9156626506024096, total=  39.3s
[CV]  n_estimators=1800, max_features=sqrt, criterion=entropy, bootstrap=False, score=0.9285714285714286, total=  39.4s
[CV] n_estimators=1800, max_features=sqrt, criterion=entropy, bootstrap=False 
[CV] n_estimators=1600, max_features=sqrt, criterion=entropy, bootstrap=True 
[CV]  n_estimators=1800, max_features=sqrt, criterion=entropy, bootstrap=False, score=0.9221556886227544, total=  40.4s
[CV] n_estimators=1600, max_features=sqrt, criterion=entropy, bootstrap=True 
[CV]  n_estimators=1800, max

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.2min


[CV]  n_estimators=1600, max_features=sqrt, criterion=entropy, bootstrap=True, score=0.9285714285714286, total=  27.4s
[CV] n_estimators=1600, max_features=sqrt, criterion=entropy, bootstrap=True 
[CV]  n_estimators=1600, max_features=sqrt, criterion=entropy, bootstrap=True, score=0.9090909090909091, total=  28.0s
[CV] n_estimators=1000, max_features=None, criterion=gini, bootstrap=False 
[CV]  n_estimators=1800, max_features=sqrt, criterion=entropy, bootstrap=False, score=0.9156626506024096, total=  44.1s
[CV] n_estimators=1000, max_features=None, criterion=gini, bootstrap=False 
[CV]  n_estimators=1600, max_features=sqrt, criterion=entropy, bootstrap=True, score=0.9156626506024096, total=  26.8s
[CV] n_estimators=1000, max_features=None, criterion=gini, bootstrap=False 
[CV]  n_estimators=1600, max_features=sqrt, criterion=entropy, bootstrap=True, score=0.9404761904761905, total=  27.9s
[CV] n_estimators=1000, max_features=None, criterion=gini, bootstrap=False 


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.7min


[CV]  n_estimators=1000, max_features=None, criterion=gini, bootstrap=False, score=0.9265536723163842, total= 6.3min
[CV] n_estimators=1000, max_features=None, criterion=gini, bootstrap=False 
[CV]  n_estimators=1000, max_features=None, criterion=gini, bootstrap=False, score=0.9101123595505618, total= 7.2min
[CV] n_estimators=1800, max_features=log2, criterion=entropy, bootstrap=False 
[CV]  n_estimators=1800, max_features=log2, criterion=entropy, bootstrap=False, score=0.8750000000000001, total=  20.7s
[CV] n_estimators=1800, max_features=log2, criterion=entropy, bootstrap=False 
[CV]  n_estimators=1000, max_features=None, criterion=gini, bootstrap=False, score=0.9060773480662984, total= 7.4min
[CV] n_estimators=1800, max_features=log2, criterion=entropy, bootstrap=False 
[CV]  n_estimators=1800, max_features=log2, criterion=entropy, bootstrap=False, score=0.9024390243902439, total=  19.4s
[CV] n_estimators=1800, max_features=log2, criterion=entropy, bootstrap=False 
[CV]  n_estimator

...

[CV]  n_estimators=1800, max_features=sqrt, criterion=gini, bootstrap=False, score=0.9411764705882353, total=  31.0s
[CV] n_estimators=1600, max_features=log2, criterion=gini, bootstrap=True 
[CV]  n_estimators=1600, max_features=log2, criterion=gini, bootstrap=True, score=0.9024390243902439, total=  12.3s
[CV] n_estimators=1600, max_features=log2, criterion=gini, bootstrap=True 
[CV]  n_estimators=1800, max_features=sqrt, criterion=gini, bootstrap=False, score=0.9221556886227544, total=  29.3s
[CV] n_estimators=1600, max_features=log2, criterion=gini, bootstrap=True 
[CV]  n_estimators=1600, max_features=log2, criterion=gini, bootstrap=True, score=0.8957055214723927, total=  12.6s
[CV] n_estimators=800, max_features=None, criterion=gini, bootstrap=False 
[CV]  n_estimators=1600, max_features=log2, criterion=gini, bootstrap=True, score=0.9146341463414633, total=  13.0s
[CV] n_estimators=800, max_features=None, criterion=gini, bootstrap=False 
[CV]  n_estimators=1600, max_features=log2,

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 17.9min


[CV]  n_estimators=800, max_features=None, criterion=gini, bootstrap=False, score=0.9060773480662984, total= 6.1min
[CV] n_estimators=1800, max_features=None, criterion=entropy, bootstrap=False 
[CV]  n_estimators=800, max_features=None, criterion=gini, bootstrap=False, score=0.9418604651162792, total= 6.7min
[CV] n_estimators=1800, max_features=None, criterion=entropy, bootstrap=False 
[CV]  n_estimators=800, max_features=None, criterion=gini, bootstrap=False, score=0.8914285714285715, total= 5.7min
[CV] n_estimators=1800, max_features=None, criterion=entropy, bootstrap=False 
[CV]  n_estimators=1800, max_features=None, criterion=entropy, bootstrap=False, score=0.8604651162790697, total=10.4min
[CV] n_estimators=1800, max_features=None, criterion=entropy, bootstrap=False 
[CV]  n_estimators=1800, max_features=None, criterion=entropy, bootstrap=False, score=0.9101123595505618, total=13.2min
[CV]  n_estimators=1800, max_features=None, criterion=entropy, bootstrap=False, score=0.87640449

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 37.5min finished


{'n_estimators': 1800, 'max_features': 'sqrt', 'criterion': 'gini', 'bootstrap': False}
0.9350213791768578


In [359]:
print('train', metrics.f1_score(y_train, model3.predict(X)))
print('test', metrics.f1_score(y_test, model3.predict(X_test)))

train 1.0
test 0.9492119089316988


In [360]:
add_metrics(best_metrics, metrics.f1_score(y_test, model3.predict(X_test)), 'baseline_3_Forest_tuned')
best_metrics

,f1
baseline_2,0.924694
baseline_3,0.963605
baseline_3_Logit_tuned,0.963855
baseline_3_Forest_tuned,0.949212


In [366]:
y_pred_prob = model2.predict_proba(X_test)[:,1] * 0.5 + model3.predict_proba(X_test)[:,1] * 0.5
y_pred = [(y_pred_prob > 0.3)]
metrics.f1_score(y_test, y_pred)

0.9639794168096055

In [370]:
add_metrics(best_metrics, metrics.f1_score(y_test, y_pred), 'baseline_4')
best_metrics

,f1
baseline_2,0.924694
baseline_3,0.963605
baseline_3_Logit_tuned,0.963855
baseline_3_Forest_tuned,0.949212
baseline_4,0.963979
